# Libraries

In [1]:
# using conda library 'ontogpt_fork'; naming irrelevant as this was previously going to be used to work on a different project.

# increase cell-width
from IPython.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import re
import numpy as np
import os
import glob
from tqdm import tqdm # progress bar tracking

# API keys:
from dotenv import load_dotenv
load_dotenv()

pd.set_option('display.max_rows', None)
pd.set_option('display.width', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

import json
import jsonschema
from jsonschema import validate

# prompts for OpenAI

import openai_secret_manager
import openai
import time
import random

import tiktoken
from typing import List, Tuple

from collections import Counter

# Import & explore dataset

In [2]:
aylien = pd.read_pickle('./datasets/Aylien/Aylien_business_articles_10k.pickle')

In [3]:
# rename id col.
aylien.rename(columns={'id': 'article_id'}, inplace=True)

In [4]:
print(len(aylien))
aylien.info()

10000
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype                  
---  ------        --------------  -----                  
 0   article_id    10000 non-null  int64                  
 1   title         10000 non-null  object                 
 2   published_at  10000 non-null  datetime64[ns, tzutc()]
 3   source        10000 non-null  object                 
 4   body          10000 non-null  object                 
dtypes: datetime64[ns, tzutc()](1), int64(1), object(3)
memory usage: 468.8+ KB


In [5]:
aylien.head(3)

,article_id,title,published_at,source,body
0,5709294677,: Nike stock jumps to lead the Dow's premarket...,2023-06-02 12:52:39+00:00,Finanzen.ch,Nike 96.41 CHF -1.99% Charts News Analysen...
1,5709292390,Renault Customers Planning Criminal Complaint ...,2023-06-02 12:52:02+00:00,MarketScreener,Financials EUR USD GBP Sales 2023 50 468 M 54 ...
2,5709276868,Mersana Therapeutics Inc. (MRSN) last month vo...,2023-06-02 12:46:55+00:00,Newsdaemon.com,Search Search Search Search Sana Meer Ju...


## Reduced subset:

In [6]:
aylien_5k = aylien[:5000].copy()

print(len(aylien_5k))
aylien_5k.head()

5000


,article_id,title,published_at,source,body
0,5709294677,: Nike stock jumps to lead the Dow's premarket...,2023-06-02 12:52:39+00:00,Finanzen.ch,Nike 96.41 CHF -1.99% Charts News Analysen...
1,5709292390,Renault Customers Planning Criminal Complaint ...,2023-06-02 12:52:02+00:00,MarketScreener,Financials EUR USD GBP Sales 2023 50 468 M 54 ...
2,5709276868,Mersana Therapeutics Inc. (MRSN) last month vo...,2023-06-02 12:46:55+00:00,Newsdaemon.com,Search Search Search Search Sana Meer Ju...
3,5709269010,Mike Novogratz Says Crypto is Lackadaisical Ri...,2023-06-02 12:46:28+00:00,Crypto Potato,"Michael Novogratz, the founder and CEO of Gala..."
4,5709271283,Another Soft Market Is Inevitable,2023-06-02 12:46:23+00:00,Carrier Management,New You can now listen to Carrier Management a...


In [7]:
aylien_5k.head()

,article_id,title,published_at,source,body
0,5709294677,: Nike stock jumps to lead the Dow's premarket...,2023-06-02 12:52:39+00:00,Finanzen.ch,Nike 96.41 CHF -1.99% Charts News Analysen...
1,5709292390,Renault Customers Planning Criminal Complaint ...,2023-06-02 12:52:02+00:00,MarketScreener,Financials EUR USD GBP Sales 2023 50 468 M 54 ...
2,5709276868,Mersana Therapeutics Inc. (MRSN) last month vo...,2023-06-02 12:46:55+00:00,Newsdaemon.com,Search Search Search Search Sana Meer Ju...
3,5709269010,Mike Novogratz Says Crypto is Lackadaisical Ri...,2023-06-02 12:46:28+00:00,Crypto Potato,"Michael Novogratz, the founder and CEO of Gala..."
4,5709271283,Another Soft Market Is Inevitable,2023-06-02 12:46:23+00:00,Carrier Management,New You can now listen to Carrier Management a...


### Convert body to list to explore.

In [8]:
article_bodies = aylien_5k['body'].tolist()

In [9]:
# observe single prompt as example.
article_bodies[0]

'Nike  96.41 CHF -1.99%  Charts  News  Analysen  Kaufen  Verkaufen Shares of Nike Inc.\nNKE jumped 3.0% to pace all of the Dow Jones Industrial Average\'s DJIA premarket gainers, putting them on track to snap a four-day losing streak. The athletic apparel giant\'s stock had actually dropped in nine of the past 10 sessions, to close Thursday at the lowest price since Dec. 20, 2022. It has lost 13.1% during the 10-day stretch, while the Dow has slipped 1.4% over the same time. During that losing stretch, Nike received a rare “sell” recommendation from Williams Trading\'s Sam Poser, who said the company\'s footwear has run “stale.” The stock\'s bounce Friday comes a day after Macy\'s Inc. M said it will bring Nike products back to certain stores and e-commerce operations in the fall, with plans to scale to additional stores in 2024.Market Pulse Stories are Rapid-fire, short news bursts on stocks and markets as they move. Visit MarketWatch.com for more information on this news.\n  Weiter z

# Exploring articles

## Investigate average length of tokens

* Important for pricing forecast and LLM API restrictions
* must use tiktoken (and not nltk), with the correct encoder.

### with tiktoken

In [10]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
#encoding = tiktoken.get_encoding("cl100k_base")

In [11]:
# fns to count strings in list of prompts:

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def count_tokens_in_list(prompt_list: list, encoding_name: str) -> list:
    """Returns a list of integers representing the number of tokens in each string in the input list."""
    token_counts = []
    for prompt in prompt_list:
        num_tokens = num_tokens_from_string(prompt, encoding_name)
        token_counts.append(num_tokens)
    return token_counts


#### Cross-check above count against count in 'prompt' col of df.

In [12]:

# Apply the num_tokens_from_string function to each row in the 'prompt' column
aylien_5k['token_count'] = aylien_5k['body'].apply(lambda x: num_tokens_from_string(x, "cl100k_base"))

# Get summary statistics
min_tokens = aylien_5k['token_count'].min()
max_tokens = aylien_5k['token_count'].max()
total_tokens = aylien_5k['token_count'].sum()
average_tokens = total_tokens / len(aylien_5k)

print(f"Number of tokens on the smallest article: {min_tokens}")
print(f"Number of tokens on the largest article: {max_tokens}")
print(f"Total number of tokens for all articles: {total_tokens}")
print(f"Average number of tokens in all articles: {average_tokens}")

Number of tokens on the smallest article: 4
Number of tokens on the largest article: 14112
Total number of tokens for all articles: 3451052
Average number of tokens in all articles: 690.2104


### Estimating max response tokens if prompt works correctly (update to reflect prompt used)

In [13]:
#num_tokens_from_string("facing risk: yes. type of risk: thisis some text for a risk type.", "cl100k_base")

num_tokens_from_string(
    
    '''
    "You will be extracting information from the provided article in the specified format."
    org: Silicon Valley Bank
    country: U.S.
    state: California
    city: Santa Clara
    industry: tech erogih eroigh erogih ergohi ergohi
    risks: political; financial; mismanagement; regulatory, eorigherogi, oeirhgoerihg , eorigheriogh
    items_sold: NA, peiorgherhg, eoirgh eroighj er, oierhg, eioprj, eighjowh, rijgio, oeirgjerj ,eoirgj
    service_provided: tech-lending; ergoiperg, eorigheropigh, oeighioerhg, oergheroig
    business_relations: Credit Suisse, eoprgje, oeirhgerhgho, woiejh, wioehr, fjkr we  d f f, wejfj r 
    '''
                       ,"cl100k_base" )

216

### Truncate tokens of long prompts

* gpt-3.5-turbo has max tokens of 4,096 tokens
* This includes prompt and response tokens combined.
* response tokens should be short due to the attempt at prompt restrictions;
    * i.e. Provide answers only in the format of <facing risk: <'yes'/'no'>. type of risk: < risk type >.> and nothing else.
* so a generous estimate of response tokens would be 400, providing gpt-3.5-turbo successfully adheres to above prompting.
* Therefore truncate prompt tokens to 3500 to be safe (truncating from the end backwards).

In [14]:
aylien_5k.head(2)

,article_id,title,published_at,source,body,token_count
0,5709294677,: Nike stock jumps to lead the Dow's premarket...,2023-06-02 12:52:39+00:00,Finanzen.ch,Nike 96.41 CHF -1.99% Charts News Analysen...,300
1,5709292390,Renault Customers Planning Criminal Complaint ...,2023-06-02 12:52:02+00:00,MarketScreener,Financials EUR USD GBP Sales 2023 50 468 M 54 ...,401


In [15]:
def num_tokens_from_string(text: str, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    return len(encoding.encode(text))

def truncate_prompt(prompt: str, encoding_name: str, max_tokens: int) -> str:
    """Truncates a text string to the specified number of tokens."""
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(prompt)[:max_tokens]
    return encoding.decode(tokens)

def truncate_prompts_in_df(df: pd.DataFrame, encoding_name: str, max_tokens: int):
    """Truncates prompts in the DataFrame if their token count exceeds the max limit."""
    df.loc[df['token_count'] > max_tokens, 'body'] = df.loc[df['token_count'] > max_tokens, 'body'].apply(
        lambda x: truncate_prompt(x, encoding_name, max_tokens)
    )
    df.loc[df['token_count'] > max_tokens, 'token_count'] = max_tokens  # Update token_count

# Apply truncation
truncate_prompts_in_df(aylien_5k, "cl100k_base", 3500)

aylien_5k.head()

,article_id,title,published_at,source,body,token_count
0,5709294677,: Nike stock jumps to lead the Dow's premarket...,2023-06-02 12:52:39+00:00,Finanzen.ch,Nike 96.41 CHF -1.99% Charts News Analysen...,300
1,5709292390,Renault Customers Planning Criminal Complaint ...,2023-06-02 12:52:02+00:00,MarketScreener,Financials EUR USD GBP Sales 2023 50 468 M 54 ...,401
2,5709276868,Mersana Therapeutics Inc. (MRSN) last month vo...,2023-06-02 12:46:55+00:00,Newsdaemon.com,Search Search Search Search Sana Meer Ju...,1289
3,5709269010,Mike Novogratz Says Crypto is Lackadaisical Ri...,2023-06-02 12:46:28+00:00,Crypto Potato,"Michael Novogratz, the founder and CEO of Gala...",513
4,5709271283,Another Soft Market Is Inevitable,2023-06-02 12:46:23+00:00,Carrier Management,New You can now listen to Carrier Management a...,1661


In [16]:
# Get truncated summary statistics:

min_tokens = aylien_5k['token_count'].min()
max_tokens = aylien_5k['token_count'].max()
total_tokens = aylien_5k['token_count'].sum()
average_tokens = total_tokens / len(aylien_5k)

print(f"Number of tokens on the smallest article: {min_tokens}")
print(f"Number of tokens on the largest article: {max_tokens}")
print(f"Total number of tokens for all articles: {total_tokens}")
print(f"Average number of tokens in all articles: {average_tokens}")
print("Number of prompts: ", len(aylien_5k))

Number of tokens on the smallest article: 4
Number of tokens on the largest article: 3500
Total number of tokens for all articles: 3363255
Average number of tokens in all articles: 672.651
Number of prompts:  5000


# Prompt structure

In [17]:
# define / restrict input size:
aylien_5k_input = aylien_5k.head(1000).copy()

In [18]:
print(len(aylien_5k_input))

1000


In [19]:
openai.api_key = os.getenv('OPENAI_KEY')

In [20]:
# model:
GPT_MODEL = "gpt-3.5-turbo"

# Retry parameters
max_retries = 3
retry_delay = 5  # in seconds

# Initialize a new column 'responses' in the dataframe
aylien_5k_input.loc[:, 'responses'] = ''

def generate_responses(input_df):
    for idx, row in tqdm(input_df.iterrows(), total=len(input_df), desc="Generating responses"):
        try:
            article = row['body']

            if article is None or article.strip() == '':
                print(f"Empty article at index {idx}. Skipping this row.")
                continue

            retries = 0
            while retries < max_retries:
                try:
                    response = openai.ChatCompletion.create(
                        model=GPT_MODEL,
                        messages=[
{"role": "system", "content": "You will be extracting information from the provided article in the specified format."},
{"role": "user", "content": '''
For the main organisation discussed in this article (if any), give me information as follows and nothing else. If there is no organiastion discussed, respond with "NA".
'''}, # In general, gpt-3.5-turbo-0301 does not pay strong attention to the system message, and therefore important instructions are often better placed in a user message.
{"role": "user", "content": '''
The desired response format is:
    ###
    org: <main organisation discussed>
    country: <country of main organisation>
    state: <state/province of main organisation>
    city: <city/town of main organisation>
    industry: <industry of the organisation>
    risks: <semi-colon-separated list of risks faced by main organisation> Summarise each risk in a phrase of up to 3 words.
    items_sold: <semicolon-separated list of items sold by main organisation>
    service_provided: <semi-colon-separated list of services provided by organisation>
    business_relations: <semi-colon-separated list of organisations with a business relation to the main organisation>
    ###
    '''},
{"role": "user", "content": f'The provided article is: {article}'},
                        ],
                        temperature=0,
                        max_tokens=400,  # max tokens in response. token limit (4096) must be < largest query + message content + max_tokens
                    )

                    response_content = response['choices'][0]['message']['content']
                    print(response_content)  # Print just the message content

                    # Add the response to the 'responses' column of the dataframe
                    input_df.loc[idx, 'responses'] = response_content

                    break  # Break out of the retry loop if successful

                except Exception as e:
                    print(f"Error generating response for prompt at index {idx}: {str(e)}")
                    print(f"Article: {article}")
                    retries += 1
                    print(f"Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)

            if retries == max_retries:
                print(f"Max retries reached for prompt at index {idx}. Skipping this row.")

            # Add a delay between requests to avoid overwhelming the API
            time.sleep(random.uniform(0.5, 1.5))

        except Exception as e:
            print(f"Error generating response for prompt at index {idx}: {str(e)}")
            print(f"Query: {article}")

## Run prompts

In [21]:
generate_responses(aylien_5k_input)

Generating responses:   0%|          | 0/1000 [00:00<?, ?it/s]

org: Nike Inc.
country: NA
state: NA
city: NA
industry: Athletic apparel
risks: Stale footwear
items_sold: Athletic apparel and footwear
service_provided: NA
business_relations: Macy's Inc.


Generating responses:   0%|          | 1/1000 [00:03<58:47,  3.53s/it]

org: RENAULT
country: NA
state: NA
city: NA
industry: Automotive
risks: Competition; Economic downturn; Supply chain disruption
items_sold: Cars; Trucks; Vans
service_provided: NA
business_relations: Nissan; Mitsubishi; Daimler


Generating responses:   0%|          | 2/1000 [00:09<1:20:45,  4.86s/it]

...

Generating responses: 100%|█████████▉| 998/1000 [1:03:15<00:07,  3.86s/it]

NA


Generating responses: 100%|█████████▉| 999/1000 [1:03:17<00:03,  3.34s/it]

org: Toyota
country: NA
state: NA
city: NA
industry: Automotive
risks: Data leak
items_sold: Cars
service_provided: NA
business_relations: NA


Generating responses: 100%|██████████| 1000/1000 [1:03:21<00:00,  3.80s/it]


In [22]:
print(len(aylien_5k))
aylien_5k_input.head()

5000


,article_id,title,published_at,source,body,token_count,responses
0,5709294677,: Nike stock jumps to lead the Dow's premarket...,2023-06-02 12:52:39+00:00,Finanzen.ch,Nike 96.41 CHF -1.99% Charts News Analysen...,300,org: Nike Inc.\ncountry: NA\nstate: NA\ncity: ...
1,5709292390,Renault Customers Planning Criminal Complaint ...,2023-06-02 12:52:02+00:00,MarketScreener,Financials EUR USD GBP Sales 2023 50 468 M 54 ...,401,org: RENAULT\ncountry: NA\nstate: NA\ncity: NA...
2,5709276868,Mersana Therapeutics Inc. (MRSN) last month vo...,2023-06-02 12:46:55+00:00,Newsdaemon.com,Search Search Search Search Sana Meer Ju...,1289,org: Mersana Therapeutics Inc.\ncountry: NA\ns...
3,5709269010,Mike Novogratz Says Crypto is Lackadaisical Ri...,2023-06-02 12:46:28+00:00,Crypto Potato,"Michael Novogratz, the founder and CEO of Gala...",513,###\norg: Galaxy Digital\ncountry: NA\nstate: ...
4,5709271283,Another Soft Market Is Inevitable,2023-06-02 12:46:23+00:00,Carrier Management,New You can now listen to Carrier Management a...,1661,NA


## Post-processing & exporting

In [23]:
data = []

def process_response(response):
    # Remove < and > characters and trailing /
    response = response.replace('<', '').replace('>', '').replace('#', '')

    item = {}
    lines = response.split('\n')
    for line in lines:
        if ': ' in line:
            key, value = line.split(': ', 1)
            key = key.lower()  # Convert key to lowercase
            if value.strip() in ["NA", "NA."]:
                item[key] = ""
            else:
                item[key] = value.strip()
    data.append(item)
    return item

# Apply the processing function to the 'responses' column and assign the result to a new column 'responses_PP'
aylien_5k_input['responses_PP'] = aylien_5k_input['responses'].apply(process_response)

# Save 'responses_PP' column as JSON
output_file = "./results/COT_aylien_business_5k_responses.json"

# Write the 'responses_PP' column data to the JSON file
with open(output_file, "w") as file:
    json.dump(aylien_5k_input['responses_PP'].tolist(), file, indent=2)

print(f"Populated schemas saved to {output_file}")

Populated schemas saved to ./results/COT_aylien_business_5k_responses.json


## Export df with responses

In [24]:
aylien_5k_input.head(2)

,article_id,title,published_at,source,body,token_count,responses,responses_PP
0,5709294677,: Nike stock jumps to lead the Dow's premarket...,2023-06-02 12:52:39+00:00,Finanzen.ch,Nike 96.41 CHF -1.99% Charts News Analysen...,300,org: Nike Inc.\ncountry: NA\nstate: NA\ncity: ...,"{'org': 'Nike Inc.', 'country': '', 'state': '..."
1,5709292390,Renault Customers Planning Criminal Complaint ...,2023-06-02 12:52:02+00:00,MarketScreener,Financials EUR USD GBP Sales 2023 50 468 M 54 ...,401,org: RENAULT\ncountry: NA\nstate: NA\ncity: NA...,"{'org': 'RENAULT', 'country': '', 'state': '',..."


In [26]:
# convert published_at to be timezone-unaware, to export to excel

aylien_5k_input['published_at'] = pd.to_datetime(aylien_5k['published_at']).dt.tz_localize(None)

# Convert 'published_at' column to 'DD-Month-YYYY' format
aylien_5k_input['published_at'] = aylien_5k['published_at'].dt.strftime('%d-%B-%Y')

In [27]:
# export to pickle in case of future use.
aylien_5k_input.to_pickle("./results/COT_aylien_business_5k_responses.pkl")

# export to excel for manual verification:
aylien_5k_input.to_excel("./results/COT_aylien_business_5k_responses.xlsx")